In [1]:
cd
use "..\Data\cleaned_cps"
describe
summarize


C:\Users\Rafael\Eco 797 - Practicum in Empirical Analysis\Relative Wage



In [2]:
replace experience =round(experience,1)
drop if experience > 39
drop if selfemployed == 1 
drop if weekly_wage == .
drop if allocated_earnings == 1 
replace top_code_wwage = 0 // We don't want to drop topcoded weekly wages
gen fulltime_fullyear = fulltime*fullyear


(1,893,739 real changes made)

(289,406 observations deleted)

(370,090 observations deleted)

(7,817 observations deleted)

(571,950 observations deleted)

(825 real changes made)



In [4]:
/* Make experience categories */ 
forval x = 5(10)35{
    gen exp`x'=0
}
replace exp5 = 5 if experience >=0 & experience <=9
replace exp15 = 15 if experience >=10 & experience <=19
replace exp25 = 25 if experience >= 20 & experience <= 29
replace exp35 = 35 if experience >= 30 & experience <= 39

/* Remake schooling categories */ 
gen slhsd = schooling == 1 // high school dropout
gen slhsg = schooling == 2 // high school graduate
gen slsmc = schooling == 3 // some college
gen slclg = schooling == 4 // college graduate
gen slclp = schooling == 5 // college plus

/* Create a quartic for experience */
rename experience experience1 // we rename experience with experience1 for clarity purposes
gen experience2 = experience1^2/100
gen experience3 = experience1^3/1000
gen experience4 = experience1^4/10000

/* Create experience-educ interactions, by three broad education categories, prefix sl for schooling */

gen e1slhsg = experience1*slhsg 
gen e1slsmc = experience1*slsmc
gen e1slclg = experience1*(slclg|slclp)

gen e2slhsg = experience2*slhsg
gen e2slsmc = experience2*slsmc
gen e2slclg = experience2*(slclg|slclp)

gen e3slhsg = experience3*slhsg
gen e3slsmc = experience3*slsmc
gen e3slclg = experience3*(slclg|slclp)

gen e4slhsg = experience4*slhsg
gen e4slsmc = experience4*slsmc
gen e4slclg = experience4*(slclg|slclp)

gen log_weekly_wage = log(real_weekly_wage)
save prelooped,replace



(1,091,064 real changes made)

(828,539 real changes made)

(722,036 real changes made)

(508,117 real changes made)























file prelooped.dta saved


In [5]:
levelsof year, local(years)

1964 1965 1966 1967 1968 1969 1970 1971 1972 1973 1974 1975 1976 1977 1978 1979 
> 1980 1981 1982 1983 1984 1985 1986 1987 1988 1989 1990 1991 1992 1993 1994 199
> 5 1996 1997 1998 1999 2000 2001 2002 2003 2004 2005 2006 2007 2008 2009 2010 2
> 011 2012 2013 2014 2015 2016 2017 2018 2019 2020


In [6]:
/*Ran regression to predict male wages across 5 education and 4 experience groups by year, according to AKK (2008)*/
foreach i in `years'{
regress log_weekly_wage slhsg slsmc slclg slclp experience1 experience2 experience3 experience4 ///
northeast midwest south black other_race e1* e2* e3* e4* [aw=asecwt] if fulltime_fullyear == 1 ///
& female == 0 & top_code_wwage == 0 & weekly_wage_less_than_sixtyseven == 0 & year == `i'

/* Create a loop to predict multiple variables at once */

foreach sl in slhsd slhsg slsmc slclg slclp {
    foreach var of varlist female black other_race northeast midwest south sl* experience* e1*  e2*  e3*  e4* {
        replace `var' = 0 in 1
    } 
        replace `sl' = 1 in 1 
        forval exp = 5(10)35 {
            replace experience1 = `exp' in 1
            replace experience2 = ((`exp')^2)/100 in 1 
            replace experience3 = ((`exp')^3)/1000 in 1 
            replace experience4 = ((`exp')^4)/10000 in 1 
            if "`sl'" == "slhsg"  {
                replace e1slhsg = experience1 in 1 
                replace e2slhsg = experience2 in 1
                replace e3slhsg = experience3 in 1 
                replace e4slhsg = experience4 in 1
            }
            if "`sl'" == "slsmc" {
                replace e1slsmc = experience1 in 1 
                replace e2slsmc = experience2 in 1
                replace e3slsmc = experience3 in 1 
                replace e4slsmc = experience4 in 1 
            }
            if "`sl'" == "slclg" {
                replace e1slclg = experience1 in 1
                replace e2slclg = experience2 in 1
                replace e3slclg = experience3 in 1
                replace e4slclg = experience4 in 1
            }
            if "`sl'" == "slclp" {
                replace e1slclg = experience1 in 1
                replace e2slclg = experience2 in 1
                replace e3slclg = experience3 in 1
                replace e4slclg = experience4 in 1
            }
            replace year = `i'
            keep female black other_race northeast midwest south sl* experience* e1*  e2*  e3*  e4* ///
            year 
            predict predicted_log_wages in 1 
            sum predicted_log_wages
            keep if _n==1
            save predicted-`sl'-exp`exp'-year`i'-m, replace
        } 
    use prelooped, clear
    } 
}
 


(sum of wgt is 25,297,308.95)

      Source |       SS           df       MS      Number of obs   =    10,362
-------------+----------------------------------   F(25, 10336)    =    183.28
       Model |  689.651065        25  27.5860426   Prob > F        =    0.0000
    Residual |  1555.74033    10,336  .150516673   R-squared       =    0.3071
-------------+----------------------------------   Adj R-squared   =    0.3055
       Total |  2245.39139    10,361  .216715702   Root MSE        =    .38796

------------------------------------------------------------------------------
log_weekly~e |      Coef.   Std. Err.      t    P>|t|     [95% Conf. Interval]
-------------+----------------------------------------------------------------
       slhsg |   .2998845   .0759841     3.95   0.000      .150941     .448828
       slsmc |   .4350153   .0799653     5.44   0.000     .2782677    .5917628
       slclg |   .7516429   .0831447     9.04   0.000     .5886631    .9146227
       slclp |   .7

(0 real changes made)
(1 real change made)
(1 real change made)
(1 real change made)
(1 real change made)
(1 real change made)
(0 real changes made)
(0 real changes made)
(1 real change made)
(0 real changes made)
(0 real changes made)
(1 real change made)
(0 real changes made)
(0 real changes made)
(1 real change made)
(0 real changes made)
(0 real changes made)
(1 real change made)
(1 real change made)
(1 real change made)
(1 real change made)
(1 real change made)
(1 real change made)
(1 real change made)
(1 real change made)
(1 real change made)
(3,127,838 real changes made)
(option xb assumed; fitted values)
(3,149,755 missing values generated)

    Variable |        Obs        Mean    Std. Dev.       Min        Max
-------------+---------------------------------------------------------
predicted_~s |          1    6.288394           .   6.288394   6.288394
(3,149,755 observations deleted)
file predicted-slsmc-exp5-year1964-m.dta saved
(1 real change made)
(1 real change made)
(1 r

    Residual |  1550.27496    10,366  .149553826   R-squared       =    0.3061
-------------+----------------------------------   Adj R-squared   =    0.3045
       Total |  2234.22864    10,391  .215015748   Root MSE        =    .38672

------------------------------------------------------------------------------
log_weekly~e |      Coef.   Std. Err.      t    P>|t|     [95% Conf. Interval]
-------------+----------------------------------------------------------------
       slhsg |     .16349   .0688975     2.37   0.018     .0284376    .2985425
       slsmc |   .3801372   .0749147     5.07   0.000       .23329    .5269845
       slclg |   .6668392   .0756137     8.82   0.000     .5186217    .8150567
       slclp |   .7584968   .0779428     9.73   0.000     .6057139    .9112798
 experience1 |   .0831421   .0170332     4.88   0.000     .0497538    .1165304
 experience2 |  -.4263036   .1524216    -2.80   0.005    -.7250793   -.1275278
 experience3 |   .1121702   .0531059     2.11   0.0

(0 real changes made)
(1 real change made)
(0 real changes made)
(0 real changes made)
(1 real change made)
(1 real change made)
(1 real change made)
(1 real change made)
(1 real change made)
(1 real change made)
(1 real change made)
(1 real change made)
(1 real change made)
(3,127,657 real changes made)
(option xb assumed; fitted values)
(3,149,755 missing values generated)

    Variable |        Obs        Mean    Std. Dev.       Min        Max
-------------+---------------------------------------------------------
predicted_~s |          1    6.296776           .   6.296776   6.296776
(3,149,755 observations deleted)
file predicted-slsmc-exp5-year1965-m.dta saved
(1 real change made)
(1 real change made)
(1 real change made)
(1 real change made)
(1 real change made)
(1 real change made)
(1 real change made)
(1 real change made)
(0 real changes made)
(option xb assumed; fitted values)

    Variable |        Obs        Mean    Std. Dev.       Min        Max
-------------+-------------

log_weekly~e |      Coef.   Std. Err.      t    P>|t|     [95% Conf. Interval]
-------------+----------------------------------------------------------------
       slhsg |   .2497072   .0495639     5.04   0.000     .1525584     .346856
       slsmc |   .2966535   .0513959     5.77   0.000     .1959138    .3973931
       slclg |   .6553049   .0523024    12.53   0.000     .5527884    .7578213
       slclp |   .7454907   .0538775    13.84   0.000     .6398869    .8510944
 experience1 |    .084779   .0122506     6.92   0.000      .060767    .1087911
 experience2 |  -.3602621   .1089721    -3.31   0.001    -.5738551   -.1466691
 experience3 |   .0701735   .0378853     1.85   0.064    -.0040843    .1444313
 experience4 |  -.0056637   .0044823    -1.26   0.206    -.0144494    .0031219
   northeast |  -.0526698    .008227    -6.40   0.000    -.0687954   -.0365442
     midwest |  -.0092821   .0081377    -1.14   0.254    -.0252326    .0066685
       south |  -.1671171   .0082084   -20.36   0.00

...

(1 real change made)
(0 real changes made)
(0 real changes made)
(0 real changes made)
(1 real change made)
(1 real change made)
(1 real change made)
(1 real change made)
(1 real change made)
(0 real changes made)
(0 real changes made)
(1 real change made)
(0 real changes made)
(0 real changes made)
(1 real change made)
(0 real changes made)
(0 real changes made)
(1 real change made)
(0 real changes made)
(0 real changes made)
(1 real change made)
(1 real change made)
(1 real change made)
(1 real change made)
(1 real change made)
(1 real change made)
(1 real change made)
(1 real change made)
(1 real change made)
(3,093,591 real changes made)
(option xb assumed; fitted values)
(3,149,755 missing values generated)

    Variable |        Obs        Mean    Std. Dev.       Min        Max
-------------+---------------------------------------------------------
predicted_~s |          1    6.274901           .   6.274901   6.274901
(3,149,755 observations deleted)
file predicted-slsmc-exp5-ye

    Residual |  7116.01298    23,431   .30370078   R-squared       =    0.3367
-------------+----------------------------------   Adj R-squared   =    0.3360
       Total |  10727.9569    23,456  .457365149   Root MSE        =    .55109

------------------------------------------------------------------------------
log_weekly~e |      Coef.   Std. Err.      t    P>|t|     [95% Conf. Interval]
-------------+----------------------------------------------------------------
       slhsg |  -.0587429   .1058625    -0.55   0.579    -.2662402    .1487545
       slsmc |  -.0255592   .1054691    -0.24   0.809    -.2322854    .1811671
       slclg |    .407877   .1032643     3.95   0.000     .2054723    .6102817
       slclp |   .6932615   .1036477     6.69   0.000     .4901052    .8964178
 experience1 |   .0061876    .030579     0.20   0.840    -.0537493    .0661244
 experience2 |   .1529077   .2975956     0.51   0.607    -.4303992    .7362146
 experience3 |  -.0824238   .1094517    -0.75   0.4

(0 real changes made)
(1 real change made)
(0 real changes made)
(0 real changes made)
(1 real change made)
(1 real change made)
(1 real change made)
(1 real change made)
(1 real change made)
(1 real change made)
(1 real change made)
(1 real change made)
(1 real change made)
(3,093,794 real changes made)
(option xb assumed; fitted values)
(3,149,755 missing values generated)

    Variable |        Obs        Mean    Std. Dev.       Min        Max
-------------+---------------------------------------------------------
predicted_~s |          1    6.293612           .   6.293612   6.293612
(3,149,755 observations deleted)
file predicted-slsmc-exp5-year2019-m.dta saved
(1 real change made)
(1 real change made)
(1 real change made)
(1 real change made)
(1 real change made)
(1 real change made)
(1 real change made)
(1 real change made)
(0 real changes made)
(option xb assumed; fitted values)

    Variable |        Obs        Mean    Std. Dev.       Min        Max
-------------+-------------

log_weekly~e |      Coef.   Std. Err.      t    P>|t|     [95% Conf. Interval]
-------------+----------------------------------------------------------------
       slhsg |  -.0530449    .101738    -0.52   0.602    -.2524597    .1463699
       slsmc |  -.0137498   .1003308    -0.14   0.891    -.2104064    .1829068
       slclg |   .4525172   .0978646     4.62   0.000     .2606947    .6443397
       slclp |   .7475397   .0982551     7.61   0.000     .5549517    .9401277
 experience1 |   .0535751   .0317896     1.69   0.092    -.0087351    .1158852
 experience2 |  -.5731509   .3205818    -1.79   0.074    -1.201517    .0552155
 experience3 |   .2440113   .1195191     2.04   0.041     .0097442    .4782785
 experience4 |  -.0333387   .0147742    -2.26   0.024    -.0622974   -.0043801
   northeast |    .017712   .0127866     1.39   0.166    -.0073509    .0427748
     midwest |  -.0216207   .0117616    -1.84   0.066    -.0446744    .0014331
       south |  -.0480757   .0104565    -4.60   0.00

(3,149,755 missing values generated)

    Variable |        Obs        Mean    Std. Dev.       Min        Max
-------------+---------------------------------------------------------
predicted_~s |          1    6.294722           .   6.294722   6.294722
(3,149,755 observations deleted)
file predicted-slsmc-exp5-year2020-m.dta saved
(1 real change made)
(1 real change made)
(1 real change made)
(1 real change made)
(1 real change made)
(1 real change made)
(1 real change made)
(1 real change made)
(0 real changes made)
(option xb assumed; fitted values)

    Variable |        Obs        Mean    Std. Dev.       Min        Max
-------------+---------------------------------------------------------
predicted_~s |          1    6.664345           .   6.664345   6.664345
(0 observations deleted)
file predicted-slsmc-exp15-year2020-m.dta saved
(1 real change made)
(1 real change made)
(1 real change made)
(1 real change made)
(1 real change made)
(1 real change made)
(1 real change made)
(1 r

In [7]:
/*Ran regression to predict female wages across 5 education and 4 experience groups by year, according to AKK (2008)*/

foreach i in `years'{
regress log_weekly_wage slhsg slsmc slclg slclp experience1 experience2 experience3 experience4 /// 
northeast midwest south black other_race e1* e2* e3* e4* [aw=asecwt] if fulltime_fullyear == 1 /// 
& female == 1 & top_code_wwage == 0 & weekly_wage_less_than_sixtyseven == 0 & year == `i'
    
foreach sl in slhsd slhsg slsmc slclg slclp {
    foreach var of varlist  black other_race northeast midwest south sl* experience* e1*  e2*  e3*  e4* {
        replace `var' = 0 in 1
    } 
        replace female = 1 
        replace `sl' = 1 in 1 
        forval exp = 5(10)35 {
            replace experience1 = `exp' in 1
            replace experience2 = ((`exp')^2)/100 in 1 
            replace experience3 = ((`exp')^3)/1000 in 1 
            replace experience4 = ((`exp')^4)/10000 in 1 
            if "`sl'" == "slhsg"  {
                replace e1slhsg = experience1 in 1 
                replace e2slhsg = experience2 in 1
                replace e3slhsg = experience3 in 1 
                replace e4slhsg = experience4 in 1
            }
            if "`sl'" == "slsmc" {
                replace e1slsmc = experience1 in 1 
                replace e2slsmc = experience2 in 1
                replace e3slsmc = experience3 in 1 
                replace e4slsmc = experience4 in 1 
            }
            if "`sl'" == "slclg" {
                replace e1slclg = experience1 in 1
                replace e2slclg = experience2 in 1
                replace e3slclg = experience3 in 1
                replace e4slclg = experience4 in 1
            }
            if "`sl'" == "slclp" {
                replace e1slclg = experience1 in 1
                replace e2slclg = experience2 in 1
                replace e3slclg = experience3 in 1
                replace e4slclg = experience4 in 1
            }
            replace year = `i'
            keep female black other_race northeast midwest south sl* experience* e1*  e2*  e3*  e4* /// 
            year
            predict predicted_log_wages in 1
            sum predicted_log_wages
            keep if _n==1
            save predicted-`sl'-exp`exp'-year`i'-f, replace
        } 
    use prelooped, clear
    }    
}


(sum of wgt is 10,034,998.95)

      Source |       SS           df       MS      Number of obs   =     4,181
-------------+----------------------------------   F(25, 4155)     =     67.23
       Model |  195.354297        25  7.81417189   Prob > F        =    0.0000
    Residual |   482.93339     4,155  .116229456   R-squared       =    0.2880
-------------+----------------------------------   Adj R-squared   =    0.2837
       Total |  678.287687     4,180  .162269782   Root MSE        =    .34092

------------------------------------------------------------------------------
log_weekly~e |      Coef.   Std. Err.      t    P>|t|     [95% Conf. Interval]
-------------+----------------------------------------------------------------
       slhsg |   .2006504   .0879123     2.28   0.023     .0282953    .3730056
       slsmc |   .3181563    .093914     3.39   0.001     .1340345     .502278
       slclg |   .5083266    .096075     5.29   0.000     .3199681    .6966851
       slclp |   .6

(1 real change made)
(1 real change made)
(1 real change made)
(1 real change made)
(1 real change made)
(0 real changes made)
(0 real changes made)
(1 real change made)
(0 real changes made)
(0 real changes made)
(1 real change made)
(0 real changes made)
(0 real changes made)
(1 real change made)
(0 real changes made)
(0 real changes made)
(1,633,331 real changes made)
(1 real change made)
(1 real change made)
(1 real change made)
(1 real change made)
(1 real change made)
(1 real change made)
(1 real change made)
(1 real change made)
(1 real change made)
(3,127,838 real changes made)
(option xb assumed; fitted values)
(3,149,755 missing values generated)

    Variable |        Obs        Mean    Std. Dev.       Min        Max
-------------+---------------------------------------------------------
predicted_~s |          1    5.934269           .   5.934269   5.934269
(3,149,755 observations deleted)
file predicted-slsmc-exp5-year1964-f.dta saved
(1 real change made)
(1 real change ma

    Residual |  522.288706     4,189  .124680999   R-squared       =    0.2678
-------------+----------------------------------   Adj R-squared   =    0.2634
       Total |  713.317391     4,214   .16927323   Root MSE        =     .3531

------------------------------------------------------------------------------
log_weekly~e |      Coef.   Std. Err.      t    P>|t|     [95% Conf. Interval]
-------------+----------------------------------------------------------------
       slhsg |   .1054459   .1184979     0.89   0.374    -.1268728    .3377647
       slsmc |   .1706663   .1243226     1.37   0.170     -.073072    .4144046
       slclg |    .503309   .1246439     4.04   0.000     .2589409    .7476771
       slclp |    .735521   .1316378     5.59   0.000      .477441     .993601
 experience1 |   .0130942   .0322853     0.41   0.685    -.0502022    .0763905
 experience2 |   .0488511    .283716     0.17   0.863    -.5073828    .6050849
 experience3 |  -.0473263   .0969741    -0.49   0.6

(0 real changes made)
(1 real change made)
(0 real changes made)
(0 real changes made)
(1,633,331 real changes made)
(1 real change made)
(1 real change made)
(1 real change made)
(1 real change made)
(1 real change made)
(1 real change made)
(1 real change made)
(1 real change made)
(1 real change made)
(3,127,657 real changes made)
(option xb assumed; fitted values)
(3,149,755 missing values generated)

    Variable |        Obs        Mean    Std. Dev.       Min        Max
-------------+---------------------------------------------------------
predicted_~s |          1    6.026368           .   6.026368   6.026368
(3,149,755 observations deleted)
file predicted-slsmc-exp5-year1965-f.dta saved
(1 real change made)
(1 real change made)
(1 real change made)
(1 real change made)
(1 real change made)
(1 real change made)
(1 real change made)
(1 real change made)
(0 real changes made)
(option xb assumed; fitted values)

    Variable |        Obs        Mean    Std. Dev.       Min        M


------------------------------------------------------------------------------
log_weekly~e |      Coef.   Std. Err.      t    P>|t|     [95% Conf. Interval]
-------------+----------------------------------------------------------------
       slhsg |   .2488344   .0719209     3.46   0.001     .1078534    .3898155
       slsmc |   .3585237   .0757231     4.73   0.000     .2100895    .5069579
       slclg |   .6413873   .0754405     8.50   0.000     .4935072    .7892674
       slclp |   .8175147   .0802141    10.19   0.000     .6602772    .9747521
 experience1 |   .0325087   .0200537     1.62   0.105     -.006801    .0718184
 experience2 |  -.1200858   .1797178    -0.67   0.504    -.4723728    .2322012
 experience3 |   .0150435   .0622381     0.24   0.809     -.106957     .137044
 experience4 |   .0000104   .0073128     0.00   0.999    -.0143242    .0143451
   northeast |  -.0297469   .0117519    -2.53   0.011    -.0527832   -.0067105
     midwest |  -.0687713   .0118193    -5.82   0.0

...

(1 real change made)
(1 real change made)
(3,093,591 real changes made)
(option xb assumed; fitted values)
(3,149,755 missing values generated)

    Variable |        Obs        Mean    Std. Dev.       Min        Max
-------------+---------------------------------------------------------
predicted_~s |          1    6.833943           .   6.833943   6.833943
(3,149,755 observations deleted)
file predicted-slclp-exp5-year2018-f.dta saved
(1 real change made)
(1 real change made)
(1 real change made)
(1 real change made)
(1 real change made)
(1 real change made)
(1 real change made)
(1 real change made)
(0 real changes made)
(option xb assumed; fitted values)

    Variable |        Obs        Mean    Std. Dev.       Min        Max
-------------+---------------------------------------------------------
predicted_~s |          1    7.078304           .   7.078304   7.078304
(0 observations deleted)
file predicted-slclp-exp15-year2018-f.dta saved
(1 real change made)
(1 real change made)
(1

-------------+---------------------------------------------------------
predicted_~s |          1     5.97588           .    5.97588    5.97588
(3,149,755 observations deleted)
file predicted-slhsg-exp5-year2019-f.dta saved
(1 real change made)
(1 real change made)
(1 real change made)
(1 real change made)
(1 real change made)
(1 real change made)
(1 real change made)
(1 real change made)
(0 real changes made)
(option xb assumed; fitted values)

    Variable |        Obs        Mean    Std. Dev.       Min        Max
-------------+---------------------------------------------------------
predicted_~s |          1    6.134596           .   6.134596   6.134596
(0 observations deleted)
file predicted-slhsg-exp15-year2019-f.dta saved
(1 real change made)
(1 real change made)
(1 real change made)
(1 real change made)
(1 real change made)
(1 real change made)
(1 real change made)
(1 real change made)
(0 real changes made)
(option xb assumed; fitted values)

    Variable |        Obs        Me

predicted_~s |          1    6.838662           .   6.838662   6.838662
(3,149,755 observations deleted)
file predicted-slclp-exp5-year2019-f.dta saved
(1 real change made)
(1 real change made)
(1 real change made)
(1 real change made)
(1 real change made)
(1 real change made)
(1 real change made)
(1 real change made)
(0 real changes made)
(option xb assumed; fitted values)

    Variable |        Obs        Mean    Std. Dev.       Min        Max
-------------+---------------------------------------------------------
predicted_~s |          1    7.055394           .   7.055394   7.055394
(0 observations deleted)
file predicted-slclp-exp15-year2019-f.dta saved
(1 real change made)
(1 real change made)
(1 real change made)
(1 real change made)
(1 real change made)
(1 real change made)
(1 real change made)
(1 real change made)
(0 real changes made)
(option xb assumed; fitted values)

    Variable |        Obs        Mean    Std. Dev.       Min        Max
-------------+---------------------

(1 real change made)
(1 real change made)
(1 real change made)
(1 real change made)
(1 real change made)
(0 real changes made)
(option xb assumed; fitted values)

    Variable |        Obs        Mean    Std. Dev.       Min        Max
-------------+---------------------------------------------------------
predicted_~s |          1    6.172492           .   6.172492   6.172492
(0 observations deleted)
file predicted-slhsg-exp15-year2020-f.dta saved
(1 real change made)
(1 real change made)
(1 real change made)
(1 real change made)
(1 real change made)
(1 real change made)
(1 real change made)
(1 real change made)
(0 real changes made)
(option xb assumed; fitted values)

    Variable |        Obs        Mean    Std. Dev.       Min        Max
-------------+---------------------------------------------------------
predicted_~s |          1     6.23719           .    6.23719    6.23719
(0 observations deleted)
file predicted-slhsg-exp25-year2020-f.dta saved
(1 real change made)
(1 real chan

(1 real change made)
(1 real change made)
(1 real change made)
(1 real change made)
(0 real changes made)
(option xb assumed; fitted values)

    Variable |        Obs        Mean    Std. Dev.       Min        Max
-------------+---------------------------------------------------------
predicted_~s |          1      7.0866           .     7.0866     7.0866
(0 observations deleted)
file predicted-slclp-exp15-year2020-f.dta saved
(1 real change made)
(1 real change made)
(1 real change made)
(1 real change made)
(1 real change made)
(1 real change made)
(1 real change made)
(1 real change made)
(0 real changes made)
(option xb assumed; fitted values)

    Variable |        Obs        Mean    Std. Dev.       Min        Max
-------------+---------------------------------------------------------
predicted_~s |          1    7.096219           .   7.096219   7.096219
(0 observations deleted)
file predicted-slclp-exp25-year2020-f.dta saved
(1 real change made)
(1 real change made)
(1 real chan

In [8]:
use predicted-slhsd-exp5-year1964-m, clear
foreach i in `years'{
    foreach sl in slhsd slhsg slsmc slclg slclp{
      forval exp = 5(10)35 {
        append using predicted-`sl'-exp`exp'-year`i'-f
          if !("`sl'"=="slhsd" & `exp'==5 & `i'== 1964) {
            append using predicted-`sl'-exp`exp'-year`i'-m  
          }
        }  
    }
}

In [9]:
sum


    Variable |        Obs        Mean    Std. Dev.       Min        Max
-------------+---------------------------------------------------------
        year |      2,280        1992    16.45556       1964       2020
   northeast |      2,280           0           0          0          0
     midwest |      2,280           0           0          0          0
       south |      2,280           0           0          0          0
       black |      2,280           0           0          0          0
-------------+---------------------------------------------------------
  other_race |      2,280           0           0          0          0
      female |      2,280          .5    .5001097          0          1
 experience1 |      2,280          20    11.18279          5         35
       slhsd |      2,280          .2    .4000877          0          1
       slhsg |      2,280          .2    .4000877          0          1
-------------+-------------------------------------------------

In [10]:
gen schooling = slhsd + 2*slhsg + 3*slsmc + 4*slclg + 5*slclp
label define schooling 1 "HS Dropout" 2 "HS Diploma" 3 "Some College" 4 "College Grad" 5 "College-Plus"
label value schooling SCHOOLING


In [11]:
rename experience1 experience_cat
recode experience_cat 5=1 15=2 25=3 35=4
sort female schooling experience_cat
keep predicted_log_wages female schooling experience_cat year 



(experience_cat: 2280 changes made)




In [13]:
save predicted_wages, replace

file predicted_wages.dta saved
